In [0]:
from keras import backend as K
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout
from keras.layers import BatchNormalization
from keras.regularizers import L1L2

In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [0]:
file_names = [
    'body_acc_x_',
    'body_acc_y_',
    'body_acc_z_',
    'body_gyro_x_',
    'body_gyro_y_',
    'body_gyro_z_',
    'total_acc_x_',
    'total_acc_y_',
    'total_acc_z_'
]

In [0]:
train_data = []
for file in file_names:
    filepath = f'dbfs:/FileStore/shared_uploads/arachure@gmu.edu/{file}train.csv'
    dataframe = spark.read.format("csv").load(filepath)
    arr = dataframe.toPandas().to_numpy()
    train_data.append(arr)

In [0]:
test_data = []
for file in file_names:
    filepath = f'dbfs:/FileStore/shared_uploads/arachure@gmu.edu/{file}test.csv'
    dataframe = spark.read.format("csv").load(filepath)
    arr = dataframe.toPandas().to_numpy()
    test_data.append(arr)

In [0]:
XTrain = np.transpose(train_data, (1, 2, 0))
XTest = np.transpose(test_data, (1, 2, 0))
print(XTrain.shape)
print(XTest.shape)

(7352, 128, 9)
(2947, 128, 9)


In [0]:
y_train = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/arachure@gmu.edu/y_train.csv")
y_test = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/arachure@gmu.edu/y_test.csv")
YTrain = y_train.toPandas()
YTest = y_test.toPandas()
YTrain = pd.get_dummies(YTrain).to_numpy()
YTest = pd.get_dummies(YTest).to_numpy()
print(YTrain.shape)
print(YTest.shape)

(7352, 6)
(2947, 6)


In [0]:
np.random.seed(9)
tf.random.set_seed(9)

epochs = 30
batch_size = 16
n_hidden = 32

def class_count(y):
    return len(set([tuple(category) for category in y]))

timesteps = len(XTrain[0])
input_dimensions = len(XTrain[0][0])
n_classes = class_count(YTrain)

# Model Initialization
model = Sequential()

# Parameter Configuration
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dimensions)))

model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                5376      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 6)                 198       
                                                                 
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [0]:
XTrain = XTrain.astype('float64')
XTest = XTest.astype('float64')

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(XTrain, YTrain, batch_size=batch_size, validation_data=(XTest, YTest), epochs=epochs)

Epoch 1/30
159/460 [=======

*** WARNING: max output size exceeded, skipping output. ***

460/460 [==============================] - 28s 60ms/step - loss: 0.1624 - accuracy: 0.9468 - val_loss: 0.5624 - val_accuracy: 0.8914


In [0]:
model.evaluate(XTest, YTest)

93/93 [==============================] - 1s 15ms/step - loss: 0.5624 - accuracy: 0.8914


[0.5624365210533142, 0.891414999961853]

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns


ACTIVITY_MAPPING = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

def confusion_matrix(Y_true, Y_pred):
    
    Y_true = pd.Series([ACTIVITY_MAPPING[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITY_MAPPING[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['Actual'], colnames=['Predicted'])

    plt.show()  
    
confusion_matrix(YTest, model.predict(XTest))

93/93 [==============================] - 2s 18ms/step


Predicted,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
Actual,,,,,,
LAYING,510,0,0,0,0,27
SITTING,0,371,99,0,2,19
STANDING,0,71,456,2,0,3
WALKING,0,0,0,462,3,31
WALKING_DOWNSTAIRS,1,1,0,3,371,44
WALKING_UPSTAIRS,0,0,0,9,5,457
